## Example sPlot  with data fitting using a simple PDF

First load the hs fit classes

In [ ]:
gROOT->ProcessLine(".x $HSCODE/hsfit/LoadFit.C+")

### Construct an sPlot object for running the fit

Set an output directory for saving weights and plots and stdout

In [ ]:
sPlot RF;
RF.SetUp().SetOutDir("/OUTPUT/DIRECTORY");

### Set an observables

These are the fit variables. In general you can fit in as many dimensions as you need by supplying more variables. You should chenge X to a variable in your tree with a sensible range.



In [ ]:
///////////////////////////////Load Variables
RF.SetUp().LoadVariable("X[-1,1]");//should be same name as variable in tree

IMPORTANT here we set the event ID variable. Each event in your tree should have a unique id with which to synchronise with weights.
    This is useful as it allows us to break the data up, perform seperate fits then combine the weights afterwards.
    The weights are not written into the input tree, rather they are stored seperately (as HS::Weights) and combined when required. This allows you to use various sets of weights when performing fits or regenerate the weights after some corrections etc.

In [ ]:
RF.SetUp().SetIDBranchName("UID");

### Create a signal PDF

The LoadSpecies Line adds the PDF Signal to the Extended Maximum Likelihood fit and creates a corresponding yield parameter Yld_Signal which will give the number of signal events in the data after the fit.
Note, I did not have to directly load this PDF I could have created more PDFs and combined them to make a more complicated PDF shape first.

Again you should change X and the ranges of the Gaussian parameters mean and sigma.

In [ ]:
//////////////////////////////Make signal PDF
RF.SetUp().FactoryPDF("Gaussian::Signal( X, meanX[-0.1,0.1], sigmaX[0.1,0.0001,0.5] )");
RF.SetUp().LoadSpeciesPDF("Signal",1); 

### Create a background PDF

Here we just use a quadratic Chebychev polynomial to describe a smooth background. You can add extra a2,a3,etc. to increase the degree of the polynomial

In [ ]:
//////////////////////////////Make background PDF
RF.SetUp().FactoryPDF("Chebychev::BG(X,{a0[-0.1,-1,1],a1[0.1,-1,1]})");
RF.SetUp().LoadSpeciesPDF("BG",1);

### Define Bins/Splits
Often you wil want to split the data into distinct bins, in invariant mass or angle for example, and perform seperate fits to each bin. Here we can set as many different variables to split the data on as we like. Here we must use the Binner class accessed via RF.Bins() to propogate the splits into data and simulated events.

The function LoadBinVars takes a variable to make bins out of which has to be in the data tree ("Y"), the number of bins to make and the limits of the variable. Alternatively you can supply the variable name, number of bins and an array of bin edges as for TAxis.

Again change "Y" for the name of the variable you want to split in.

In [ ]:
RF.Bins().LoadBinVar("Y",5,3,4);

### Load Data


The treename is FinalTree and the data files are OUTPUT.root.
Change these to whatever your data is stored as.

In [ ]:
///////////////////////////Load Data
RF.LoadData("FinalTree","OUTPUT.root");

Turn on javascript root for interactive plots (even when saved as html file!)

In [ ]:
%jsroot

### Run the fit.
The test output will be redirected to your specified output directory (see code cell 2) as file logRooFit.txt. This will give full details of the fit.
Here we will just report the final result, the weights and the plots of fit PDF overlayed on the data as well as the fit residuals and pulls.

There are different options for running. 
To just run standard use Here::Go and the fits will just be performed within this interactive session, sequentially in the case of bins.

Here::One(&RF,i) will run one singal fit from the different bins produce with LoadBinVars, where i gives the index of the bin

Use Proof::Go(&RF,N) and the fits will be run in parallel via PROOF with each split/bin running on a different core, where N = number of cores to use.
    

In [ ]:
Here::Go(&RF);
//Here::One(&RF,3);
//Proof::Go(&RF,1);

### Draw some weighted variables

I can now use the resulting weights to make sPlots of the other variables in the tree, disentangling signal from background. 
I use the DrawWeighted function which takes standard TTree::Draw option for first argument and the weight species with the second (i.e. here I could plot Signal or BG) 

Here V1 and V2 would be other variables in the tree.

In [ ]:
auto* can=new TCanvas;
can->Divide(2,1);
can->cd(1);
RF.DrawWeighted("V1>>V1(100,0,10)","Signal");

can->cd(2);
RF.DrawWeighted("V2>>V2(100,0,10)","Signal");

can->Draw();